In [ ]:
import opentrons.execute
from opentrons import protocol_api

# Setup

## Inputs

In [ ]:
total_nuclease_free_water_volume = 500
total_pcr_master_mix_volume = 500
init_volume_per_frag_well = 50

per_reaction_nuclease_free_water_volume = 20
per_reaction_pcr_mastermix_volume = 20
per_reaction_frag_volume = 15

## General deck setup

In [ ]:
protocol = opentrons.execute.get_protocol_api("2.17")
protocol.home()

In [ ]:
# Load tip racks
tip_rack_20 = protocol.load_labware('opentrons_96_tiprack_20ul', 9)
tip_rack_300 = protocol.load_labware('opentrons_96_tiprack_300ul', 6)

In [1]:
# Load pipette tips
p20 = protocol.load_instrument(instrument_name='p20_single_gen2', mount="right", tip_racks = [tip_rack_20])
p300 = protocol.load_instrument(instrument_name='p300_single', mount='left', tip_racks=[tip_rack_300])

In [ ]:
# Load well plates
frag_plate = protocol.load_labware("corning_96_wellplate_360ul_flat", 1)
qc_plate = protocol.load_labware("corning_96_wellplate_360ul_flat", 2)

### Add reagents to well plate

In [ ]:
# Define liquids (optional)
suspended_dna = protocol.define_liquid(
    name="Suspended DNA",
    description="Oligo fragments suspended in buffer",
    display_color="#a8329d",
)

In [ ]:
frag_wells = frag_plate.wells()[0:19]  # check that layout matches frag plate; currently goes A1, B1, C1,...

for well in frag_wells:
    well.load_liquid(suspended_dna, volume=50)

## Load modules

In [ ]:
thermocycler = protocol.load_module("thermocyclerModuleV1", 7)
thermocycler_plate = thermocycler.load_labware("nest_96_wellplate_100ul_pcr_full_skirt")

In [ ]:
temp_module = protocol.load_module("temperatureModuleV1", 3)
temp_tubes = temp_module.load_labware("opentrons_24_aluminumblock_generic_2ml_screwcap")

### Adding reagents to temperature controller

In [2]:
# Define liquids (optional)
nuclease_free_water = protocol.define_liquid(
    name="Nuclease free water",
    description="Nuclease free water",
    display_color="#0000FF",
)

pcr_mastermix = protocol.define_liquid(
    name="PCR mastermix",
    description="PCR mastermix",
    display_color="#00FF00",
)

NameError: name 'protocol' is not defined

In [ ]:
# Define reagents
water_well = temp_tubes.wells_by_name()["A1"]
pcr_mastermix_well = temp_tubes.wells_by_name()["A2"]
waste_well = temp_tubes.wells_by_name()["A3"]

# Load liquids into the corresponding wells
water_well.load_liquid(liquid=nuclease_free_water, volume=200)
pcr_mastermix_well.load_liquid(liquid=pcr_mastermix, volume=200)

# Run protocol

### Assembly Operations Mapping

The `assembly_operations` dictionary provides a mapping of the different assembly stages and the corresponding well combinations for each stage. This mapping allows us to automate the pipetting operations required for the polymerase chain assembly (PCA) protocol.

The dictionary is structured as follows:
- Keys at the top level represent the assembly stages ("1", "2", "3", "4", "5").
- Within each stage, keys represent the destination well (e.g., "A1", "B1", "C1").
- Values are lists containing the wells or fragments to be combined in the destination well.

For example:
- In Assembly Stage 1, Well A1 contains fragments 1 and 2, Well A2 contains fragments 3 and 4, and so on.
- In Assembly Stage 2, Well B1 contains products from wells A1 and A2, Well B2 contains products from wells A3 and A4, and so on.

This pattern continues until the final assembled product is achieved in Well E1 during Operation 5.

In [3]:
assembly_operations = {
    "1": {
        "A1": ["1", "2"],
        "A2": ["3", "4"],
        "A3": ["5", "6"],
        "A4": ["7", "8"],
        "A5": ["9", "10"],
        "A6": ["11", "12"],
        "A7": ["13", "14"],
        "A8": ["15", "16"],
        "A9": ["17", "18"],
        "A10": ["19", "20"]
    },
    "2": {
        "B1": ["A1", "A2"],
        "B2": ["A3", "A4"],
        "B3": ["A5", "A6"],
        "B4": ["A7", "A8"],
        "B5": ["A9", "A10"]
    },
    "3": {
        "C1": ["B1", "B2"],
        "C2": ["B3", "B4"],
        "C3": ["B5"]
    },
    "4": {
        "D1": ["C1", "C2"],
        "D2": ["C3"]
    },
    "5": {
        "E1": ["D1", "D2"]
    }
}

In [ ]:
thermocycler.open_lid()
temp_module.set_temperature(celsius=4)

 ### Assembly stage 1

In [4]:
def add_pcr_master_mix(protocol):
    # Add PCR master mix to each reaction well
    p300.pick_up_tip()
    for well in thermocycler_plate.wells()[:10]:
        p300.aspirate(25, pcr_mastermix_well.top(z=-18))  # check z offsets
        p300.dispense(25, well.top(z=2))
    p300.drop_tip()
    
add_pcr_master_mix(protocol)
# todo: consider making the PCR wells more central for even heating

NameError: name 'protocol' is not defined

# Utility commands

In [ ]:
def print_deck_layout(protocol):
    for slot, item in protocol.deck.items():
        if item:
            print(f"Slot {slot}: {item}")
        else:
            print(f"Slot {slot}: Empty")
            
print_deck_layout(protocol)